<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Justin-Hamilton-Modeling-Contribution" data-toc-modified-id="Justin-Hamilton-Modeling-Contribution-0">Justin Hamilton Modeling Contribution</a></span></li><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span><ul class="toc-item"><li><span><a href="#Project-Goal" data-toc-modified-id="Project-Goal-1.1">Project Goal</a></span></li><li><span><a href="#Business-Problem-Statement" data-toc-modified-id="Business-Problem-Statement-1.2">Business Problem Statement</a></span></li><li><span><a href="#Analytic-Problem" data-toc-modified-id="Analytic-Problem-1.3">Analytic Problem</a></span></li><li><span><a href="#Model-Notebook-Purpose" data-toc-modified-id="Model-Notebook-Purpose-1.4">Model Notebook Purpose</a></span></li></ul></li><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-2">Data Preparation</a></span></li><li><span><a href="#Modeling-Process" data-toc-modified-id="Modeling-Process-3">Modeling Process</a></span><ul class="toc-item"><li><span><a href="#Building-the-SARIMA-Model" data-toc-modified-id="Building-the-SARIMA-Model-3.1">Building the SARIMA Model</a></span></li><li><span><a href="#Predicting-for-a-Year-on-a-Daily-Basis-For-Model" data-toc-modified-id="Predicting-for-a-Year-on-a-Daily-Basis-For-Model-3.2">Predicting for a Year on a Daily Basis For Model</a></span></li><li><span><a href="#Making-the-Prediction-Table-More-Easy-to-Read" data-toc-modified-id="Making-the-Prediction-Table-More-Easy-to-Read-3.3">Making the Prediction Table More Easy to Read</a></span></li><li><span><a href="#Allowing-the-User-to-Input-Data-About-a-Store-and-Predict" data-toc-modified-id="Allowing-the-User-to-Input-Data-About-a-Store-and-Predict-3.4">Allowing the User to Input Data About a Store and Predict</a></span></li></ul></li><li><span><a href="#Model-Performance" data-toc-modified-id="Model-Performance-4">Model Performance</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-5">Results</a></span></li><li><span><a href="#Group-Member-Contributions" data-toc-modified-id="Group-Member-Contributions-6">Group Member Contributions</a></span></li></ul></div>

# Justin Hamilton Modeling Contribution

# Introduction

## Project Goal

Maverik will be able to implement the model created in order to predict daily sales for diesel fuel, unleaded fuel, merchandise, and food sales for the entire first year of a store's operation. This model needs to surpass the accuracy metrics used by the current Maverik model in predicting these 4 product sales. 


## Business Problem Statement

Maverik, a convenience store and gas station chain with 300+ locations in the Western United States, aims to enhance its ability to assess the return on investment (ROI) for new stores, which are opened at an average rate of 30 per year. To achieve this, they require a new model capable of generating daily sales projections for merchandise, food sales, and total gallons sold for diesel and unleaded fuel for the entire first year of a store's operation. This data-driven approach will enable Maverik to closely monitor the performance of new stores, make informed decisions, and promptly address any under performing outlets. Maverik desires to know the daily sales for each of the 4 main products diesel fuel, unleaded fuel, merchandise, and food sales. Having these daily predictions allows for better planning for seasonality and setting realistic goals and expectations for what each new store can bring in revenue for the year.

## Analytic Problem

The target variables is specifically daily sales for the diesel, unleaded fuel, merchandise, and food services. Represented in the time_series_data_msba.csv sets of time series sales data where the variables

* daily_yoy_ndt.total_inside_sales:	Inside sales, everything that isn't made at the store
* daily_yoy_ndt.total_food_service:	Food service sales, everything that is made at the store
* diesel:	Diesel gallons sold
* unleaded:	all non-diesel gallons

The main level of accuracy that we will use to determine the performance of this model is RMSE and R Squared.

* Predict the daily sales for each 4 products diesel, unleaded fuel, merchandise, and food service for each store for the following year based on the time series data.

* Use a Prophet method to see the daily sales predictions for each of the 4 product categories for a new store.

The success of this project will be measured by its capacity to accurately predict sales metrics and its usability for future stores. Our team will be judging our model based on R-squared and RSME results. Since our target variables are numeric, we will explore which data features are significant via a linear regression model and then apply those features to a variety of data modeling techniques including a time series model. In this notebook we will be exploring and cleaning the data. Some of the questions we would like to answer are the following:


## Model Notebook Purpose

This notebook walks through the creating a model that will be used for future stores based on qualitative information to estimate the sales of the  store for the in store food service, merchandise, unleaded gasoline sales, and diesel fuel. The notebook explores the process of developing a time forecasting model, the performance of each model, and the interpretation of the model results.

The primary objectives of this Modeling notebook are as follows:

* Model Development: Develop a predictive model that will allow a user to insert qualitative information about a new store, then output the predicted sales for in store food service, merchandise, diesel fuel, and unleaded gasoline fuel. The diesel and unleaded fuel are gallons sold, while the in store food service and merchandise are in dollars.
* Performance: The model developed is to have a better performing RMSE and R Squared than the current model used by Maverik.
* Reproduction: The model that is developed here should easily be able to be used by teams at Maverik. 
* Interpretation: The model should give us useful insights into the predictive amount sold so that Maverik can plan and anticipate for a new stores opening based on the inputted categorical information. 


In [1]:
# Installing the Packages for the Modeling

# !pip install prophet
# !pip install Cython
# ! pip install fbprophet
# !pip uninstall numpy
# !pip install numpy
# !pip install --upgrade numpy
# !pip install statsmodels==0.13.0
# ! pip install pmdarima




# Data Preparation

In [2]:
# Importing the necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from pmdarima.arima import auto_arima

# Importing the data files
time = pd.read_csv('C:/Users/jusha/Documents/IS 6813 MSBA Final Capstone Maverick Project Folder/Individual Contributions/time_series_data_msba.csv')
qual = pd.read_csv('C:/Users/jusha/Documents/IS 6813 MSBA Final Capstone Maverick Project Folder/Individual Contributions/qualitative_data_msba.csv')


In [3]:
# Check for Input 
print(time.head())
print(qual.head())


   Unnamed: 0 capital_projects.soft_opening_date calendar.calendar_day_date  \
0           1                         2022-06-14                 2022-06-17   
1           2                         2022-06-14                 2022-06-22   
2           3                         2022-06-14                 2022-06-23   
3           4                         2022-06-14                 2022-06-26   
4           5                         2022-06-14                 2022-06-27   

   calendar.fiscal_week_id_for_year calendar.day_of_week  \
0                                25               Friday   
1                                25            Wednesday   
2                                25             Thursday   
3                                26               Sunday   
4                                26               Monday   

  calendar_information.holiday calendar_information.type_of_day  \
0                         NONE                          WEEKDAY   
1                         NONE

We see that the data has been loaded properly in order for us to use into our modeling. 

In [4]:
# Gathering Basic Info of Sheets
print(time.describe())
print(qual.describe())


         Unnamed: 0  calendar.fiscal_week_id_for_year  \
count  13908.000000                      13908.000000   
mean    6954.500000                         26.501079   
std     4015.038107                         14.998715   
min        1.000000                          1.000000   
25%     3477.750000                         14.000000   
50%     6954.500000                         26.000000   
75%    10431.250000                         39.000000   
max    13908.000000                         52.000000   

       daily_yoy_ndt.total_inside_sales  daily_yoy_ndt.total_food_service  \
count                      13908.000000                      13908.000000   
mean                        2846.537988                        759.922326   
std                          981.299870                        341.578220   
min                            0.000000                          0.000000   
25%                         2181.156250                        521.087875   
50%                     

Using the describe() function allows us to see the different patterns associated with in our EDA and is a good reference to use as we go through the modeling portion. These ranges allow us to reflect on our model later to make sure the predictions fit well into our historical data and make sense. 

In [5]:
# Check for Null Values 

print(time.isnull().sum())
print(qual.isnull().sum())


Unnamed: 0                            0
capital_projects.soft_opening_date    0
calendar.calendar_day_date            0
calendar.fiscal_week_id_for_year      0
calendar.day_of_week                  0
calendar_information.holiday          0
calendar_information.type_of_day      0
daily_yoy_ndt.total_inside_sales      0
daily_yoy_ndt.total_food_service      0
diesel                                0
unleaded                              0
site_id_msba                          0
dtype: int64
Unnamed: 0                                  0
open_year                                   0
square_feet                                 0
front_door_count                            0
years_since_last_project                    0
parking_spaces                              0
lottery                                     0
freal                                       0
bonfire_grill                               0
pizza                                       0
cinnabon                                    0
g

Recalling again that although the values in the data set


* rv_lanes_layout                            14
* rv_lanes_stack_type                        14
* hi_flow_lanes_layout                       15
* hi_flow_lanes_stack_type                   15
* hi_flow_rv_lanes_layout                    14
* hi_flow_rv_lanes_stack_type                14

are inputted as "NONE" or "NA" this does not mean that they are missing.  This means that the store does not provide these services, which is still information that we need to have kept in our model for predicting the sales amount. Therefore, the datasets are cleaned and do not have any missing values.

In [6]:
# Merge the dataframes based on site_id_msba so that the model takes into account all information of the store and previous sales.
combined_data = pd.merge(time, qual, on="site_id_msba")

display(combined_data)

,Unnamed: 0_x,capital_projects.soft_opening_date,calendar.calendar_day_date,calendar.fiscal_week_id_for_year,calendar.day_of_week,calendar_information.holiday,calendar_information.type_of_day,daily_yoy_ndt.total_inside_sales,daily_yoy_ndt.total_food_service,diesel_x,...,hi_flow_lanes_fueling_positions_2,rv_lanes_fueling_positions_2,hi_flow_rv_lanes_layout,hi_flow_rv_lanes_stack_type,non_24_hour,self_check_out,mens_toilet_count,mens_urinal_count,womens_toilet_count,womens_sink_count
0,1,2022-06-14,2022-06-17,25,Friday,NONE,WEEKDAY,2168.2920,861.6930,722.7745,...,5,4,Combo,HF/RV,No,Yes,1,1,2,2
1,2,2022-06-14,2022-06-22,25,Wednesday,NONE,WEEKDAY,2051.5635,808.0275,730.4850,...,5,4,Combo,HF/RV,No,Yes,1,1,2,2
2,3,2022-06-14,2022-06-23,25,Thursday,NONE,WEEKDAY,2257.5000,966.4410,895.7970,...,5,4,Combo,HF/RV,No,Yes,1,1,2,2
3,4,2022-06-14,2022-06-26,26,Sunday,NONE,WEEKEND,1520.5925,542.3250,584.2900,...,5,4,Combo,HF/RV,No,Yes,1,1,2,2
4,5,2022-06-14,2022-06-27,26,Monday,NONE,WEEKDAY,1897.6930,771.4525,852.2605,...,5,4,Combo,HF/RV,No,Yes,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13537,13904,2021-01-12,2021-12-28,52,Tuesday,NONE,WEEKDAY,2984.0300,864.2795,1609.7830,...,4,6,Stack,HF/RV,No,Yes,2,2,6,2
13538,13905,2021-01-12,2022-01-01,1,Saturday,New Year's Day,WEEKEND,2483.4145,597.8315,731.2935,...,4,6,Stack,HF/RV,No,Yes,2,2,6,2
13539,13906,2021-01-12,2022-01-04,1,Tuesday,NONE,WEEKDAY,3169.6875,970.4100,1851.6470,...,4,6,Stack,HF/RV,No,Yes,2,2,6,2
13540,13907,2021-01-12,2022-01-07,2,Friday,NONE,WEEKDAY,3559.5805,1072.4175,2104.9070,...,4,6,Stack,HF/RV,No,Yes,2,2,6,2


We merge the data sets here based on the site_msba_id variable so that we can use one data set for the model to train against. We see here that the merging has added the unnecessary columns of Unnamed 0_X and 0_Y. We need to remove these so the modeling processes does not confuse them as being variables.

In [7]:
# Remove the columns as not being predictors. These are the Unnamed Columns because they simply were listed the types of data a long the  sheet.
columns_to_remove = ['Unnamed: 0_x', 'Unnamed: 0_y']
combined_data.drop(columns=columns_to_remove, inplace=True)
display(combined_data)


with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(combined_data.dtypes)

,capital_projects.soft_opening_date,calendar.calendar_day_date,calendar.fiscal_week_id_for_year,calendar.day_of_week,calendar_information.holiday,calendar_information.type_of_day,daily_yoy_ndt.total_inside_sales,daily_yoy_ndt.total_food_service,diesel_x,unleaded,...,hi_flow_lanes_fueling_positions_2,rv_lanes_fueling_positions_2,hi_flow_rv_lanes_layout,hi_flow_rv_lanes_stack_type,non_24_hour,self_check_out,mens_toilet_count,mens_urinal_count,womens_toilet_count,womens_sink_count
0,2022-06-14,2022-06-17,25,Friday,NONE,WEEKDAY,2168.2920,861.6930,722.7745,1425.1020,...,5,4,Combo,HF/RV,No,Yes,1,1,2,2
1,2022-06-14,2022-06-22,25,Wednesday,NONE,WEEKDAY,2051.5635,808.0275,730.4850,1436.2740,...,5,4,Combo,HF/RV,No,Yes,1,1,2,2
2,2022-06-14,2022-06-23,25,Thursday,NONE,WEEKDAY,2257.5000,966.4410,895.7970,1594.3725,...,5,4,Combo,HF/RV,No,Yes,1,1,2,2
3,2022-06-14,2022-06-26,26,Sunday,NONE,WEEKEND,1520.5925,542.3250,584.2900,1124.9280,...,5,4,Combo,HF/RV,No,Yes,1,1,2,2
4,2022-06-14,2022-06-27,26,Monday,NONE,WEEKDAY,1897.6930,771.4525,852.2605,1640.2540,...,5,4,Combo,HF/RV,No,Yes,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13537,2021-01-12,2021-12-28,52,Tuesday,NONE,WEEKDAY,2984.0300,864.2795,1609.7830,2492.5180,...,4,6,Stack,HF/RV,No,Yes,2,2,6,2
13538,2021-01-12,2022-01-01,1,Saturday,New Year's Day,WEEKEND,2483.4145,597.8315,731.2935,1564.0170,...,4,6,Stack,HF/RV,No,Yes,2,2,6,2
13539,2021-01-12,2022-01-04,1,Tuesday,NONE,WEEKDAY,3169.6875,970.4100,1851.6470,2767.5970,...,4,6,Stack,HF/RV,No,Yes,2,2,6,2
13540,2021-01-12,2022-01-07,2,Friday,NONE,WEEKDAY,3559.5805,1072.4175,2104.9070,2868.2325,...,4,6,Stack,HF/RV,No,Yes,2,2,6,2


capital_projects.soft_opening_date          object
calendar.calendar_day_date                  object
calendar.fiscal_week_id_for_year             int64
calendar.day_of_week                        object
calendar_information.holiday                object
calendar_information.type_of_day            object
daily_yoy_ndt.total_inside_sales           float64
daily_yoy_ndt.total_food_service           float64
diesel_x                                   float64
unleaded                                   float64
site_id_msba                                 int64
open_year                                    int64
square_feet                                  int64
front_door_count                             int64
years_since_last_project                     int64
parking_spaces                               int64
lottery                                     object
freal                                       object
bonfire_grill                               object
pizza                          

Now that we have the time series and the qualitative data formed into one data sheet, we now need to account for our variables that are factors in the dataset. In our model we do want to use the dates of sell for each of our 4 target variables as predictors because this will allow the model to build patterns of seasonality and make accurate predictions based on the range when we ask the model to predict for the following year. 

The Columns that need to be Factored


   * Categorical Predictors to be Factored:
        * 'lottery'
        * 'freal'
        * 'bonfire_grill'
        * 'pizza'
        * 'cinnabon'
        * 'godfather_s_pizza'
        * 'ethanol_free'
        * 'diesel'
        * 'hi_flow_lanes'
        * 'rv_lanes'
        * 'hi_flow_rv_lanes'
        * 'def'
        * 'cat_scales'
        * 'car_wash'
        * 'ev_charging'
        * 'rv_dumps'
        * 'propane'
        * 'traditional_forecourt_layout'
        * 'traditional_forecourt_stack_type'
        * 'rv_lanes_layout'
        * 'rv_lanes_stack_type'
        * 'hi_flow_lanes_layout'
        * 'hi_flow_lanes_stack_type'
        * 'hi_flow_rv_lanes_layout'
        * 'hi_flow_rv_lanes_stack_type'
        * 'non_24_hour'
        * 'self_check_out'
        * calendar.day_of_week	
        * calendar_information.holiday	
        * calendar_information.type_of_day
        * capital_projects.soft_opening_date	
        * calendar.calendar_day_date


In [8]:
# List of Categorical Predictors to be Label Encoded
columns_to_label_encode = [
    'lottery', 'freal', 'bonfire_grill', 'pizza', 'cinnabon', 'godfather_s_pizza',
    'ethanol_free', 'diesel_y', 'hi_flow_lanes', 'rv_lanes', 'hi_flow_rv_lanes',
    'def', 'cat_scales', 'car_wash', 'ev_charging', 'rv_dumps', 'propane',
    'traditional_forecourt_layout', 'traditional_forecourt_stack_type',
    'rv_lanes_layout', 'rv_lanes_stack_type', 'hi_flow_lanes_layout',
    'hi_flow_lanes_stack_type', 'hi_flow_rv_lanes_layout', 'hi_flow_rv_lanes_stack_type',
    'non_24_hour', 'self_check_out',
    'calendar.day_of_week', 'calendar_information.holiday', 'calendar_information.type_of_day'
    ,'calendar.calendar_day_date' , 'capital_projects.soft_opening_date'
]


# ,     'calendar.calendar_day_date' , 'capital_projects.soft_opening_date'

label_encoder = LabelEncoder()

# Create a dictionary to store encoding keys for each column
encoding_keys = {}

for column in columns_to_label_encode:
    combined_data[column] = label_encoder.fit_transform(combined_data[column])
    encoding_keys[column] = {label: category for label, category in enumerate(label_encoder.classes_)}

# Display encoding keys for each column
for column, encoding_key in encoding_keys.items():
    print(f"Encoding key for {column}:")
    print(encoding_key)
    print("\n")

# Display all columns and their data types
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(combined_data.dtypes)

# Display column names
print(combined_data.columns)

# Display the entire DataFrame
display(combined_data)

Encoding key for lottery:
{0: 'No', 1: 'Yes'}


Encoding key for freal:
{0: 'No', 1: 'Yes'}


Encoding key for bonfire_grill:
{0: 'No', 1: 'Yes'}


Encoding key for pizza:
{0: 'No', 1: 'Yes'}


Encoding key for cinnabon:
{0: 'No', 1: 'Yes'}


Encoding key for godfather_s_pizza:
{0: 'No'}


Encoding key for ethanol_free:
{0: 'No', 1: 'Yes'}


Encoding key for diesel_y:
{0: 'Yes'}


Encoding key for hi_flow_lanes:
{0: 'No', 1: 'Yes'}


Encoding key for rv_lanes:
{0: 'No', 1: 'Yes'}


Encoding key for hi_flow_rv_lanes:
{0: 'No', 1: 'Yes'}


Encoding key for def:
{0: 'No', 1: 'Yes'}


Encoding key for cat_scales:
{0: 'No', 1: 'Yes'}


Encoding key for car_wash:
{0: 'No'}


Encoding key for ev_charging:
{0: 'No'}


Encoding key for rv_dumps:
{0: 'No', 1: 'Yes'}


Encoding key for propane:
{0: 'No', 1: 'Yes'}


Encoding key for traditional_forecourt_layout:
{0: 'In-Line', 1: 'Stack'}


Encoding key for traditional_forecourt_stack_type:
{0: 'Extra-Large', 1: 'Large', 2: 'None'}


Encoding key

,capital_projects.soft_opening_date,calendar.calendar_day_date,calendar.fiscal_week_id_for_year,calendar.day_of_week,calendar_information.holiday,calendar_information.type_of_day,daily_yoy_ndt.total_inside_sales,daily_yoy_ndt.total_food_service,diesel_x,unleaded,...,hi_flow_lanes_fueling_positions_2,rv_lanes_fueling_positions_2,hi_flow_rv_lanes_layout,hi_flow_rv_lanes_stack_type,non_24_hour,self_check_out,mens_toilet_count,mens_urinal_count,womens_toilet_count,womens_sink_count
0,25,521,25,0,16,0,2168.2920,861.6930,722.7745,1425.1020,...,5,4,0,0,0,0,1,1,2,2
1,25,526,25,6,16,0,2051.5635,808.0275,730.4850,1436.2740,...,5,4,0,0,0,0,1,1,2,2
2,25,527,25,4,16,0,2257.5000,966.4410,895.7970,1594.3725,...,5,4,0,0,0,0,1,1,2,2
3,25,530,26,3,16,1,1520.5925,542.3250,584.2900,1124.9280,...,5,4,0,0,0,0,1,1,2,2
4,25,531,26,1,16,0,1897.6930,771.4525,852.2605,1640.2540,...,5,4,0,0,0,0,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13537,0,350,52,5,16,0,2984.0300,864.2795,1609.7830,2492.5180,...,4,6,2,0,0,0,2,2,6,2
13538,0,354,1,2,17,1,2483.4145,597.8315,731.2935,1564.0170,...,4,6,2,0,0,0,2,2,6,2
13539,0,357,1,5,16,0,3169.6875,970.4100,1851.6470,2767.5970,...,4,6,2,0,0,0,2,2,6,2
13540,0,360,2,0,16,0,3559.5805,1072.4175,2104.9070,2868.2325,...,4,6,2,0,0,0,2,2,6,2


Here we factor our categorical variables into encodings. We use One Hot Encoding to encode the dates because the SARIMA model that we are going to use requires the data types be a number or factored variable type. This encoding works because the model still uses the dates as the references to each of the dates and then will use those values to make predictions for the units spent on each of our four major indicators. This output will also be useful as when we use our predictive model we can input the categorical values that match the descriptions of the store and date that it was opened to make accurate predictions to the forecast of the sales of the 4 target variables.

# Modeling Process

In [16]:
# Separating Predictors of the Values
predictors = combined_data[['capital_projects.soft_opening_date', 'calendar.calendar_day_date', 'calendar.fiscal_week_id_for_year', 'calendar.day_of_week', 'calendar_information.holiday', 'calendar_information.type_of_day', 'open_year', 'square_feet', 'front_door_count', 'years_since_last_project', 'parking_spaces', 'lottery', 'freal', 'bonfire_grill', 'pizza', 'cinnabon', 'godfather_s_pizza', 'ethanol_free', 'diesel_y', 'hi_flow_lanes', 'rv_lanes', 'hi_flow_rv_lanes', 'def', 'cat_scales', 'car_wash', 'ev_charging', 'rv_dumps', 'propane', 'x1_mile_pop', 'x1_mile_emp', 'x1_mile_income', 'x1_2_mile_pop', 'x1_2_mile_emp', 'x1_2_mile_income', 'x5_min_pop', 'x5_min_emp', 'x5_min_inc', 'x7_min_pop', 'x7_min_emp', 'x7_min_inc', 'traditional_forecourt_fueling_positions', 'traditional_forecourt_layout', 'traditional_forecourt_stack_type', 'rv_lanes_fueling_positions', 'rv_lanes_layout', 'rv_lanes_stack_type', 'hi_flow_lanes_fueling_positions', 'hi_flow_lanes_layout', 'hi_flow_lanes_stack_type', 'hi_flow_lanes_fueling_positions_2', 'rv_lanes_fueling_positions_2', 'hi_flow_rv_lanes_layout', 'hi_flow_rv_lanes_stack_type', 'non_24_hour', 'self_check_out', 'mens_toilet_count', 'mens_urinal_count', 'womens_toilet_count', 'womens_sink_count']]

# Separating Target Variables
target_variables = combined_data[['daily_yoy_ndt.total_inside_sales', 'daily_yoy_ndt.total_food_service', 'diesel_x', 'unleaded']]


In [17]:
# Splitting the Data into Test and Training Sets

predictors_train, predictors_test, target_variables_train, target_variables_test = train_test_split(predictors, target_variables, test_size=0.2, random_state=42)


Because this is a time series model, we do use the dates as the predictors. The dates are important to use as predictors because they will allow us to see the seasonality patterns and then make forecast off of these date values when we build the model and have it make predictions. We also include the categorical data of the store because these aspects may be making a difference in explaining the variation of the prediction of our model.  We split the train-test set 80-20 because the 80-20 models tend to perform better for RMSE and R2 and explain the variance in the models better than the 70-30 split model. 

## Building the SARIMA Model

In [18]:
# Fit SARIMA models for each of the target variables
sarima_models = {}

for target_variable in target_variables_train.columns:
    # Perform a grid search for the best hyperparameters
    best_model = auto_arima(
        target_variables_train[target_variable],
        exogenous=predictors_train,
        seasonal=True,
        m= 7,
        stepwise=True,
        trace=True,
        start_p=0, start_q=0,
        max_p=3, max_q=3,
        start_P=0, start_Q=0,
        max_P=3, max_Q=3,
        suppress_warnings=True,
        error_action="ignore"
    )
    
    # Retrieve the best hyperparameters
    best_p, best_d, best_q = best_model.get_params()['order']
    best_P, best_D, best_Q, best_S = best_model.get_params()['seasonal_order']
    
    # Fit the SARIMA model with the best hyperparameters
    sarima_model = SARIMAX(target_variables_train[target_variable], exog=predictors_train, order=(best_p, best_d, best_q), seasonal_order=(best_P, best_D, best_Q, best_S))
    sarima_results = sarima_model.fit()
    sarima_models[target_variable] = sarima_results

# Make predictions for test data
forecasted_values = {}

for target_variable, model in sarima_models.items():
    forecasted_values[target_variable] = model.get_forecast(steps=len(target_variables_test), exog=predictors_test)

# Model Evaluation
for target_variable in target_variables_test.columns:
    # Get the predicted values from the SARIMA results
    forecasted_values_for_target = forecasted_values[target_variable].predicted_mean
    # Convert the predicted values to a NumPy array for evaluation
    forecasted_values_for_target = np.array(forecasted_values_for_target)

    mse = mean_squared_error(target_variables_test[target_variable], forecasted_values_for_target)
    rmse = np.sqrt(mse)
    r2 = r2_score(target_variables_test[target_variable], forecasted_values_for_target)
    
    # Round the output values to 2 decimal places
    mse = round(mse, 2)
    rmse = round(rmse, 2)
    r2 = round(r2, 2)

    print(f'Mean Squared Error for {target_variable}: {mse}')
    print(f'Root Mean Squared Error for {target_variable}: {rmse}')
    print(f'R-squared for {target_variable}: {r2}')


Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[7] intercept   : AIC=180221.667, Time=0.10 sec
 ARIMA(1,0,0)(1,0,0)[7] intercept   : AIC=180222.176, Time=2.21 sec
 ARIMA(0,0,1)(0,0,1)[7] intercept   : AIC=180219.033, Time=0.79 sec
 ARIMA(0,0,0)(0,0,0)[7]             : AIC=204374.695, Time=0.05 sec
 ARIMA(0,0,1)(0,0,0)[7] intercept   : AIC=180223.541, Time=0.31 sec
 ARIMA(0,0,1)(1,0,1)[7] intercept   : AIC=180221.393, Time=5.93 sec
 ARIMA(0,0,1)(0,0,2)[7] intercept   : AIC=180221.033, Time=1.36 sec
 ARIMA(0,0,1)(1,0,0)[7] intercept   : AIC=180224.661, Time=1.68 sec
 ARIMA(0,0,1)(1,0,2)[7] intercept   : AIC=180223.023, Time=6.71 sec
 ARIMA(0,0,0)(0,0,1)[7] intercept   : AIC=180217.204, Time=0.43 sec
 ARIMA(0,0,0)(1,0,1)[7] intercept   : AIC=180219.725, Time=5.59 sec
 ARIMA(0,0,0)(0,0,2)[7] intercept   : AIC=180219.203, Time=1.04 sec
 ARIMA(0,0,0)(1,0,0)[7] intercept   : AIC=180222.972, Time=0.59 sec
 ARIMA(0,0,0)(1,0,2)[7] intercept   : AIC=180221.204, Time=4.92 sec
 ARIM

C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[7] intercept   : AIC=157317.359, Time=0.13 sec
 ARIMA(1,0,0)(1,0,0)[7] intercept   : AIC=157318.547, Time=1.54 sec
 ARIMA(0,0,1)(0,0,1)[7] intercept   : AIC=157316.165, Time=0.72 sec
 ARIMA(0,0,0)(0,0,0)[7]             : AIC=176647.597, Time=0.06 sec
 ARIMA(0,0,1)(0,0,0)[7] intercept   : AIC=157319.324, Time=0.29 sec
 ARIMA(0,0,1)(1,0,1)[7] intercept   : AIC=157323.860, Time=5.01 sec
 ARIMA(0,0,1)(0,0,2)[7] intercept   : AIC=157318.117, Time=1.36 sec
 ARIMA(0,0,1)(1,0,0)[7] intercept   : AIC=157320.480, Time=1.21 sec
 ARIMA(0,0,1)(1,0,2)[7] intercept   : AIC=162590.286, Time=14.50 sec
 ARIMA(0,0,0)(0,0,1)[7] intercept   : AIC=157314.193, Time=0.42 sec
 ARIMA(0,0,0)(1,0,1)[7] intercept   : AIC=157320.520, Time=5.87 sec
 ARIMA(0,0,0)(0,0,2)[7] intercept   : AIC=157316.145, Time=1.08 sec
 ARIMA(0,0,0)(1,0,0)[7] intercept   : AIC=157318.485, Time=0.69 sec
 ARIMA(0,0,0)(1,0,2)[7] intercept   : AIC=157485.799, Time=12.41 sec
 AR

C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[7] intercept   : AIC=197488.475, Time=0.10 sec
 ARIMA(1,0,0)(1,0,0)[7] intercept   : AIC=197481.627, Time=0.74 sec
 ARIMA(0,0,1)(0,0,1)[7] intercept   : AIC=197477.468, Time=0.70 sec
 ARIMA(0,0,0)(0,0,0)[7]             : AIC=202783.240, Time=0.08 sec
 ARIMA(0,0,1)(0,0,0)[7] intercept   : AIC=197490.472, Time=0.24 sec
 ARIMA(0,0,1)(1,0,1)[7] intercept   : AIC=197479.451, Time=4.96 sec
 ARIMA(0,0,1)(0,0,2)[7] intercept   : AIC=197479.141, Time=1.38 sec
 ARIMA(0,0,1)(1,0,0)[7] intercept   : AIC=197482.716, Time=0.70 sec
 ARIMA(0,0,1)(1,0,2)[7] intercept   : AIC=197481.143, Time=8.84 sec
 ARIMA(0,0,0)(0,0,1)[7] intercept   : AIC=197475.480, Time=0.44 sec
 ARIMA(0,0,0)(1,0,1)[7] intercept   : AIC=197477.483, Time=2.92 sec
 ARIMA(0,0,0)(0,0,2)[7] intercept   : AIC=197477.154, Time=1.17 sec
 ARIMA(0,0,0)(1,0,0)[7] intercept   : AIC=197481.200, Time=0.52 sec
 ARIMA(0,0,0)(1,0,2)[7] intercept   : AIC=197479.165, Time=9.33 sec
 ARIM

C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[7] intercept   : AIC=181323.524, Time=0.09 sec
 ARIMA(1,0,0)(1,0,0)[7] intercept   : AIC=181324.729, Time=1.43 sec
 ARIMA(0,0,1)(0,0,1)[7] intercept   : AIC=181324.313, Time=0.66 sec
 ARIMA(0,0,0)(0,0,0)[7]             : AIC=201201.598, Time=0.08 sec
 ARIMA(0,0,0)(1,0,0)[7] intercept   : AIC=181325.358, Time=1.33 sec
 ARIMA(0,0,0)(0,0,1)[7] intercept   : AIC=181324.476, Time=0.44 sec
 ARIMA(0,0,0)(1,0,1)[7] intercept   : AIC=181330.213, Time=4.42 sec
 ARIMA(1,0,0)(0,0,0)[7] intercept   : AIC=181323.284, Time=0.23 sec
 ARIMA(1,0,0)(0,0,1)[7] intercept   : AIC=181324.238, Time=0.84 sec
 ARIMA(1,0,0)(1,0,1)[7] intercept   : AIC=181329.409, Time=7.38 sec
 ARIMA(2,0,0)(0,0,0)[7] intercept   : AIC=181325.121, Time=0.31 sec
 ARIMA(1,0,1)(0,0,0)[7] intercept   : AIC=181325.288, Time=0.36 sec
 ARIMA(0,0,1)(0,0,0)[7] intercept   : AIC=181323.358, Time=0.25 sec
 ARIMA(2,0,1)(0,0,0)[7] intercept   : AIC=181327.121, Time=0.71 sec
 ARIM

C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836

Mean Squared Error for daily_yoy_ndt.total_inside_sales: 390298.85
Root Mean Squared Error for daily_yoy_ndt.total_inside_sales: 624.74
R-squared for daily_yoy_ndt.total_inside_sales: 0.59
Mean Squared Error for daily_yoy_ndt.total_food_service: 36517.96
Root Mean Squared Error for daily_yoy_ndt.total_food_service: 191.1
R-squared for daily_yoy_ndt.total_food_service: 0.68
Mean Squared Error for diesel_x: 896962.86
Root Mean Squared Error for diesel_x: 947.08
R-squared for diesel_x: 0.8
Mean Squared Error for unleaded: 275211.57
Root Mean Squared Error for unleaded: 524.61
R-squared for unleaded: 0.73


Here we use the for loop to create an individual SARIMA model for each of our 4 target variables. We use the SARIMA model because this model takes into account the seasonality. We then perform the grid search to find the best performing hyper parameters of each model for the hyper parameters. 

We then print out each of the RMSE and R2 of the models to evaluate their performance and make sure there is no over fitting occurring. 

We can set M = Number of Periods for Seasonality. Example: Because this is daily data over a year M would = 365. However, because the M = 365 parameter requires too much memory to complete we substitute it with M = 7. This is because with M = 7 the memory space is sufficient. M = 7 allows the model to find the patterns that occur on weekly basis over the course of the year. This could be changed to M = 30 for monthly, M = 90 for each of the 4 seasons, ETC. Although anything above M = 10 tends to have memory error of insufficient space.

The performance of each of the models is discussed in a below section. 

## Predicting for a Year on a Daily Basis For Model

In [19]:
from datetime import datetime

# Get today's date
current_date = pd.Timestamp(datetime.now())

# Generate a date range for the next year (365 days) starting from today
prediction_dates = pd.date_range(start=current_date, periods=365)

# Create exogenous variables for the next year
exog_next_year = pd.DataFrame(index=prediction_dates, columns=predictors.columns)

# Fill exog_next_year with the same data for each day
for col in exog_next_year.columns:
    exog_next_year[col] = predictors[col].values[0]

# Initialize a dictionary to store daily sales projections
daily_projections = {}

# Specify the target variables for which you want to calculate daily projections
target_variables = combined_data[['daily_yoy_ndt.total_inside_sales', 'daily_yoy_ndt.total_food_service', 'diesel_x', 'unleaded']]

# Make predictions for each of the target variables
for target_variable in target_variables:
    # Use the SARIMA model to forecast sales for the next year
    forecasted_values = sarima_models[target_variable].get_forecast(steps=365, exog=exog_next_year)

    # Extract the predicted values
    predicted_values = forecasted_values.predicted_mean
    
    # Round the predicted values to 2 decimal places
    predicted_values = predicted_values.round(2)

    # Store the daily projections in the dictionary and reset the index
    daily_projections[target_variable] = predicted_values
    daily_projections[target_variable].index = prediction_dates

# Print the daily projections for each target variable
for target_variable, daily_projection in daily_projections.items():
    print(f"Daily Projections for {target_variable}:")
    print(daily_projection)


Daily Projections for daily_yoy_ndt.total_inside_sales:
2023-11-03 21:44:26.735940    2814.86
2023-11-04 21:44:26.735940    2816.14
2023-11-05 21:44:26.735940    2817.68
2023-11-06 21:44:26.735940    2816.33
2023-11-07 21:44:26.735940    2816.62
                               ...   
2024-10-28 21:44:26.735940    2813.21
2024-10-29 21:44:26.735940    2813.21
2024-10-30 21:44:26.735940    2813.21
2024-10-31 21:44:26.735940    2813.21
2024-11-01 21:44:26.735940    2813.21
Freq: D, Name: predicted_mean, Length: 365, dtype: float64
Daily Projections for daily_yoy_ndt.total_food_service:
2023-11-03 21:44:26.735940    957.11
2023-11-04 21:44:26.735940    957.14
2023-11-05 21:44:26.735940    956.96
2023-11-06 21:44:26.735940    957.20
2023-11-07 21:44:26.735940    957.06
                               ...  
2024-10-28 21:44:26.735940    957.18
2024-10-29 21:44:26.735940    957.18
2024-10-30 21:44:26.735940    957.18
2024-10-31 21:44:26.735940    957.18
2024-11-01 21:44:26.735940    957.18
Freq

C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\jusha\Ne

Here we test our model and make predictions for the inside sales, food service, diesel, and unleaded gasoline. This forecasting model then prints out 365 predicted values for each sales units for the remainder of the year. This model will take today's current date and then make the prediction based on the current date and moving forward. This model is specifically designed to forecast the estimated sales of a store and does not take into account the specific qualitative variables of the individual store as that did not match with the business objective. This model however can be run across a CSV file and predict the sales of the 4 target variables over 365 days for that store. While these forecasts may have some varying units because of the RMSE measurement for each of the predictor values. Assuming that the qualitative and time series data sheets would be updated and reloaded as necessary, this model then predicts the average forecasts for the unleaded, diesel, food service, and in store merchandise sales of an average Maverik store. 



## Making the Prediction Table More Easy to Read

In [20]:
# Create a DataFrame from daily_projections
daily_projections_df = pd.DataFrame(daily_projections)

# Set the date index to a column
daily_projections_df['date'] = daily_projections_df.index

# Reorder the columns to match the desired order
desired_order = [
    'daily_yoy_ndt.total_inside_sales', 'daily_yoy_ndt.total_food_service', 'diesel_x', 'unleaded'
]
daily_projections_df = daily_projections_df[desired_order]

# Export the DataFrame to a CSV file
daily_projections_df.to_csv('daily_projections.csv', index=False)

# Display the top 10 rows
display(daily_projections_df.head(300))


,daily_yoy_ndt.total_inside_sales,daily_yoy_ndt.total_food_service,diesel_x,unleaded
2023-11-03 21:44:26.735940,2814.86,957.11,1163.92,2838.21
2023-11-04 21:44:26.735940,2816.14,957.14,1159.95,2838.09
2023-11-05 21:44:26.735940,2817.68,956.96,1161.89,2838.09
2023-11-06 21:44:26.735940,2816.33,957.20,1170.32,2838.09
2023-11-07 21:44:26.735940,2816.62,957.06,1160.99,2838.09
...,...,...,...,...
2024-08-24 21:44:26.735940,2813.21,957.18,1164.25,2838.09
2024-08-25 21:44:26.735940,2813.21,957.18,1164.25,2838.09
2024-08-26 21:44:26.735940,2813.21,957.18,1164.25,2838.09
2024-08-27 21:44:26.735940,2813.21,957.18,1164.25,2838.09


Here we print the model into a table that is easier to read. When we have M = 7 for sake of the memory being sufficient to run on my machine we see that the top 7 values are only predicted for and then after that the values become redundant. When we adjust the M = 365 for the model then there will be a more robust estimate of each of the 365 days in the year. We can learn here that our model does accomplish the business objective of predicting the amount sold for a store in each day based on previous historical data. When we insert the model into a more powerful computer, it will account for M = 365, then it will generate a different value for each of the 365 days in our prediction sheet rather than what we are only seeing now as the first 7 only change, because we have given our seasonality of M = 7.

A possibility to increase the effectiveness and use of this model and planning for Maverik would be to create a User Interface that would allow the Maverik to answer a series of questions about the qualitative information of Maverik stores, then insert their responses to values for those stores for the model to then predict the exact individual store forecast sales for each new unit. The current model takes into account several different stores and the output is estimated average sales over the dates for each store rather than the model create location specific sales. Although the individual store may not be completely accurate forecasting because this could potentially lead to over fit values or values that do not make sense because the model is trained on several stores locations and historical values rather than an individual stores. But creating that User Interface could make the input of qualitative information specific to that store interesting to view for a future project rather than how our model now trains on the macro level, it would be interesting to take it to the micro exact store level. 

We do see that the model does predict for a years prediction of each of the inputs. We see that the units are predicted in place for each of the stores. 

Example: This model predicts that an average store on November 4, 2023 will have a 

* 2,816.14 units (dollars) sold in Inside Sales
* 957.14 units (dollars) sold in Food Service
* 1,159.95 units (gallons) sold in Diesel Sales	
* 2,838.09 units (gallons) sold in Unleaded Fuel

## Allowing the User to Input Data About a Store and Predict

Our previous model is built for Maverik to replace our data set that we have used, with their CSV files for predictions. However, this code below is meant for a more User Friendly Situation where variables can be put into easily and predict for the store based on the inserted information. 



For reference of the user input to input correct information, these are the types of encodings that occur and how the user should input the values for these specific encodings. 

Encoding key for lottery:
{0: 'No', 1: 'Yes'}


Encoding key for freal:
{0: 'No', 1: 'Yes'}


Encoding key for bonfire_grill:
{0: 'No', 1: 'Yes'}


Encoding key for pizza:
{0: 'No', 1: 'Yes'}


Encoding key for cinnabon:
{0: 'No', 1: 'Yes'}


Encoding key for godfather_s_pizza:
{0: 'No'}


Encoding key for ethanol_free:
{0: 'No', 1: 'Yes'}


Encoding key for diesel_y:
{0: 'Yes'}


Encoding key for hi_flow_lanes:
{0: 'No', 1: 'Yes'}


Encoding key for rv_lanes:
{0: 'No', 1: 'Yes'}


Encoding key for hi_flow_rv_lanes:
{0: 'No', 1: 'Yes'}


Encoding key for def:
{0: 'No', 1: 'Yes'}


Encoding key for cat_scales:
{0: 'No', 1: 'Yes'}


Encoding key for car_wash:
{0: 'No'}


Encoding key for ev_charging:
{0: 'No'}


Encoding key for rv_dumps:
{0: 'No', 1: 'Yes'}


Encoding key for propane:
{0: 'No', 1: 'Yes'}


Encoding key for traditional_forecourt_layout:
{0: 'In-Line', 1: 'Stack'}


Encoding key for traditional_forecourt_stack_type:
{0: 'Extra-Large', 1: 'Large', 2: 'None'}


Encoding key for rv_lanes_layout:
{0: 'In-Line', 1: 'Stack', 2: nan}


Encoding key for rv_lanes_stack_type:
{0: 'HF/RV', 1: 'None', 2: nan}


Encoding key for hi_flow_lanes_layout:
{0: 'Combo', 1: 'Stack', 2: nan}


Encoding key for hi_flow_lanes_stack_type:
{0: 'HF/RV', 1: nan}


Encoding key for hi_flow_rv_lanes_layout:
{0: 'Combo', 1: 'In-Line', 2: 'Stack', 3: nan}


Encoding key for hi_flow_rv_lanes_stack_type:
{0: 'HF/RV', 1: 'None', 2: nan}


Encoding key for non_24_hour:
{0: 'No'}


Encoding key for self_check_out:
{0: 'Yes'}


Encoding key for calendar.day_of_week:
{0: 'Friday', 1: 'Monday', 2: 'Saturday', 3: 'Sunday', 4: 'Thursday', 5: 'Tuesday', 6: 'Wednesday'}


Encoding key for calendar_information.holiday:
{0: "All Saint's Day", 1: 'Ascension', 2: 'Christmas Day', 3: 'Christmas Eve', 4: 'Columbus Day', 5: 'Easter', 6: "Father's Day", 7: 'Flag Day', 8: 'Good Friday', 9: 'Halloween Day', 10: 'Independence Day', 11: 'Labor Day', 12: "Lincoln's Birthday", 13: 'Martin Luther King Day', 14: 'Memorial Day', 15: "Mother's Day", 16: 'NONE', 17: "New Year's Day", 18: "New Year's Eve", 19: 'Palm Sunday', 20: "President's Day", 21: "Saint Patrick's Day", 22: "Saint Valentine's Day", 23: 'Thanksgiving Day', 24: "Veteran's Day", 25: "Washington's Birthday"}


Encoding key for calendar_information.type_of_day:
{0: 'WEEKDAY', 1: 'WEEKEND'}


Encoding key for calendar.calendar_day_date:
{0: '2021-01-12', 1: '2021-01-13', 2: '2021-01-14', 3: '2021-01-15', 4: '2021-01-16', 5: '2021-01-17', 6: '2021-01-18', 7: '2021-01-19', 8: '2021-01-20', 9: '2021-01-21', 10: '2021-01-22', 11: '2021-01-23', 12: '2021-01-24', 13: '2021-01-25', 14: '2021-01-26', 15: '2021-01-27', 16: '2021-01-28', 17: '2021-01-29', 18: '2021-01-30', 19: '2021-01-31', 20: '2021-02-01', 21: '2021-02-02', 22: '2021-02-03', 23: '2021-02-04', 24: '2021-02-05', 25: '2021-02-06', 26: '2021-02-07', 27: '2021-02-08', 28: '2021-02-09', 29: '2021-02-10', 30: '2021-02-11', 31: '2021-02-12', 32: '2021-02-13', 33: '2021-02-14', 34: '2021-02-15', 35: '2021-02-16', 36: '2021-02-17', 37: '2021-02-18', 38: '2021-02-19', 39: '2021-02-20', 40: '2021-02-21', 41: '2021-02-22', 42: '2021-02-23', 43: '2021-02-24', 44: '2021-02-25', 45: '2021-02-26', 46: '2021-02-27', 47: '2021-02-28', 48: '2021-03-01', 49: '2021-03-02', 50: '2021-03-03', 51: '2021-03-04', 52: '2021-03-05', 53: '2021-03-06', 54: '2021-03-07', 55: '2021-03-08', 56: '2021-03-09', 57: '2021-03-10', 58: '2021-03-11', 59: '2021-03-12', 60: '2021-03-13', 61: '2021-03-14', 62: '2021-03-15', 63: '2021-03-16', 64: '2021-03-17', 65: '2021-03-18', 66: '2021-03-19', 67: '2021-03-20', 68: '2021-03-21', 69: '2021-03-22', 70: '2021-03-23', 71: '2021-03-24', 72: '2021-03-25', 73: '2021-03-26', 74: '2021-03-27', 75: '2021-03-28', 76: '2021-03-29', 77: '2021-03-30', 78: '2021-03-31', 79: '2021-04-01', 80: '2021-04-02', 81: '2021-04-03', 82: '2021-04-04', 83: '2021-04-05', 84: '2021-04-06', 85: '2021-04-07', 86: '2021-04-08', 87: '2021-04-09', 88: '2021-04-10', 89: '2021-04-11', 90: '2021-04-12', 91: '2021-04-13', 92: '2021-04-14', 93: '2021-04-15', 94: '2021-04-16', 95: '2021-04-17', 96: '2021-04-18', 97: '2021-04-19', 98: '2021-04-20', 99: '2021-04-21', 100: '2021-04-22', 101: '2021-04-23', 102: '2021-04-24', 103: '2021-04-25', 104: '2021-04-26', 105: '2021-04-27', 106: '2021-04-28', 107: '2021-04-29', 108: '2021-04-30', 109: '2021-05-01', 110: '2021-05-02', 111: '2021-05-03', 112: '2021-05-04', 113: '2021-05-05', 114: '2021-05-06', 115: '2021-05-07', 116: '2021-05-08', 117: '2021-05-09', 118: '2021-05-10', 119: '2021-05-11', 120: '2021-05-12', 121: '2021-05-13', 122: '2021-05-14', 123: '2021-05-15', 124: '2021-05-16', 125: '2021-05-17', 126: '2021-05-18', 127: '2021-05-19', 128: '2021-05-20', 129: '2021-05-21', 130: '2021-05-22', 131: '2021-05-23', 132: '2021-05-24', 133: '2021-05-25', 134: '2021-05-26', 135: '2021-05-27', 136: '2021-05-28', 137: '2021-05-29', 138: '2021-05-30', 139: '2021-05-31', 140: '2021-06-01', 141: '2021-06-02', 142: '2021-06-03', 143: '2021-06-04', 144: '2021-06-05', 145: '2021-06-06', 146: '2021-06-07', 147: '2021-06-08', 148: '2021-06-09', 149: '2021-06-10', 150: '2021-06-11', 151: '2021-06-12', 152: '2021-06-13', 153: '2021-06-14', 154: '2021-06-15', 155: '2021-06-16', 156: '2021-06-17', 157: '2021-06-18', 158: '2021-06-19', 159: '2021-06-20', 160: '2021-06-21', 161: '2021-06-22', 162: '2021-06-23', 163: '2021-06-24', 164: '2021-06-25', 165: '2021-06-26', 166: '2021-06-27', 167: '2021-06-28', 168: '2021-06-29', 169: '2021-06-30', 170: '2021-07-01', 171: '2021-07-02', 172: '2021-07-03', 173: '2021-07-04', 174: '2021-07-05', 175: '2021-07-06', 176: '2021-07-07', 177: '2021-07-08', 178: '2021-07-09', 179: '2021-07-10', 180: '2021-07-11', 181: '2021-07-12', 182: '2021-07-13', 183: '2021-07-14', 184: '2021-07-15', 185: '2021-07-16', 186: '2021-07-17', 187: '2021-07-18', 188: '2021-07-19', 189: '2021-07-20', 190: '2021-07-21', 191: '2021-07-22', 192: '2021-07-23', 193: '2021-07-24', 194: '2021-07-25', 195: '2021-07-26', 196: '2021-07-27', 197: '2021-07-28', 198: '2021-07-29', 199: '2021-07-30', 200: '2021-07-31', 201: '2021-08-01', 202: '2021-08-02', 203: '2021-08-03', 204: '2021-08-04', 205: '2021-08-05', 206: '2021-08-06', 207: '2021-08-07', 208: '2021-08-08', 209: '2021-08-09', 210: '2021-08-10', 211: '2021-08-11', 212: '2021-08-12', 213: '2021-08-13', 214: '2021-08-14', 215: '2021-08-15', 216: '2021-08-16', 217: '2021-08-17', 218: '2021-08-18', 219: '2021-08-19', 220: '2021-08-20', 221: '2021-08-21', 222: '2021-08-22', 223: '2021-08-23', 224: '2021-08-24', 225: '2021-08-25', 226: '2021-08-26', 227: '2021-08-27', 228: '2021-08-28', 229: '2021-08-29', 230: '2021-08-30', 231: '2021-08-31', 232: '2021-09-01', 233: '2021-09-02', 234: '2021-09-03', 235: '2021-09-04', 236: '2021-09-05', 237: '2021-09-06', 238: '2021-09-07', 239: '2021-09-08', 240: '2021-09-09', 241: '2021-09-10', 242: '2021-09-11', 243: '2021-09-12', 244: '2021-09-13', 245: '2021-09-14', 246: '2021-09-15', 247: '2021-09-16', 248: '2021-09-17', 249: '2021-09-18', 250: '2021-09-19', 251: '2021-09-20', 252: '2021-09-21', 253: '2021-09-22', 254: '2021-09-23', 255: '2021-09-24', 256: '2021-09-25', 257: '2021-09-26', 258: '2021-09-27', 259: '2021-09-28', 260: '2021-09-29', 261: '2021-09-30', 262: '2021-10-01', 263: '2021-10-02', 264: '2021-10-03', 265: '2021-10-04', 266: '2021-10-05', 267: '2021-10-06', 268: '2021-10-07', 269: '2021-10-08', 270: '2021-10-09', 271: '2021-10-10', 272: '2021-10-11', 273: '2021-10-12', 274: '2021-10-13', 275: '2021-10-14', 276: '2021-10-15', 277: '2021-10-16', 278: '2021-10-17', 279: '2021-10-18', 280: '2021-10-19', 281: '2021-10-20', 282: '2021-10-21', 283: '2021-10-22', 284: '2021-10-23', 285: '2021-10-24', 286: '2021-10-25', 287: '2021-10-26', 288: '2021-10-27', 289: '2021-10-28', 290: '2021-10-29', 291: '2021-10-30', 292: '2021-10-31', 293: '2021-11-01', 294: '2021-11-02', 295: '2021-11-03', 296: '2021-11-04', 297: '2021-11-05', 298: '2021-11-06', 299: '2021-11-07', 300: '2021-11-08', 301: '2021-11-09', 302: '2021-11-10', 303: '2021-11-11', 304: '2021-11-12', 305: '2021-11-13', 306: '2021-11-14', 307: '2021-11-15', 308: '2021-11-16', 309: '2021-11-17', 310: '2021-11-18', 311: '2021-11-19', 312: '2021-11-20', 313: '2021-11-21', 314: '2021-11-22', 315: '2021-11-23', 316: '2021-11-24', 317: '2021-11-25', 318: '2021-11-26', 319: '2021-11-27', 320: '2021-11-28', 321: '2021-11-29', 322: '2021-11-30', 323: '2021-12-01', 324: '2021-12-02', 325: '2021-12-03', 326: '2021-12-04', 327: '2021-12-05', 328: '2021-12-06', 329: '2021-12-07', 330: '2021-12-08', 331: '2021-12-09', 332: '2021-12-10', 333: '2021-12-11', 334: '2021-12-12', 335: '2021-12-13', 336: '2021-12-14', 337: '2021-12-15', 338: '2021-12-16', 339: '2021-12-17', 340: '2021-12-18', 341: '2021-12-19', 342: '2021-12-20', 343: '2021-12-21', 344: '2021-12-22', 345: '2021-12-23', 346: '2021-12-24', 347: '2021-12-25', 348: '2021-12-26', 349: '2021-12-27', 350: '2021-12-28', 351: '2021-12-29', 352: '2021-12-30', 353: '2021-12-31', 354: '2022-01-01', 355: '2022-01-02', 356: '2022-01-03', 357: '2022-01-04', 358: '2022-01-05', 359: '2022-01-06', 360: '2022-01-07', 361: '2022-01-08', 362: '2022-01-09', 363: '2022-01-10', 364: '2022-01-11', 365: '2022-01-12', 366: '2022-01-13', 367: '2022-01-14', 368: '2022-01-15', 369: '2022-01-16', 370: '2022-01-17', 371: '2022-01-18', 372: '2022-01-19', 373: '2022-01-20', 374: '2022-01-21', 375: '2022-01-22', 376: '2022-01-23', 377: '2022-01-24', 378: '2022-01-25', 379: '2022-01-26', 380: '2022-01-27', 381: '2022-01-28', 382: '2022-01-29', 383: '2022-01-30', 384: '2022-01-31', 385: '2022-02-01', 386: '2022-02-02', 387: '2022-02-03', 388: '2022-02-04', 389: '2022-02-05', 390: '2022-02-06', 391: '2022-02-07', 392: '2022-02-08', 393: '2022-02-09', 394: '2022-02-10', 395: '2022-02-11', 396: '2022-02-12', 397: '2022-02-13', 398: '2022-02-14', 399: '2022-02-15', 400: '2022-02-16', 401: '2022-02-17', 402: '2022-02-18', 403: '2022-02-19', 404: '2022-02-20', 405: '2022-02-21', 406: '2022-02-22', 407: '2022-02-23', 408: '2022-02-24', 409: '2022-02-25', 410: '2022-02-26', 411: '2022-02-27', 412: '2022-02-28', 413: '2022-03-01', 414: '2022-03-02', 415: '2022-03-03', 416: '2022-03-04', 417: '2022-03-05', 418: '2022-03-06', 419: '2022-03-07', 420: '2022-03-08', 421: '2022-03-09', 422: '2022-03-10', 423: '2022-03-11', 424: '2022-03-12', 425: '2022-03-13', 426: '2022-03-14', 427: '2022-03-15', 428: '2022-03-16', 429: '2022-03-17', 430: '2022-03-18', 431: '2022-03-19', 432: '2022-03-20', 433: '2022-03-21', 434: '2022-03-22', 435: '2022-03-23', 436: '2022-03-24', 437: '2022-03-25', 438: '2022-03-26', 439: '2022-03-27', 440: '2022-03-28', 441: '2022-03-29', 442: '2022-03-30', 443: '2022-03-31', 444: '2022-04-01', 445: '2022-04-02', 446: '2022-04-03', 447: '2022-04-04', 448: '2022-04-05', 449: '2022-04-06', 450: '2022-04-07', 451: '2022-04-08', 452: '2022-04-09', 453: '2022-04-10', 454: '2022-04-11', 455: '2022-04-12', 456: '2022-04-13', 457: '2022-04-14', 458: '2022-04-15', 459: '2022-04-16', 460: '2022-04-17', 461: '2022-04-18', 462: '2022-04-19', 463: '2022-04-20', 464: '2022-04-21', 465: '2022-04-22', 466: '2022-04-23', 467: '2022-04-24', 468: '2022-04-25', 469: '2022-04-26', 470: '2022-04-27', 471: '2022-04-28', 472: '2022-04-29', 473: '2022-04-30', 474: '2022-05-01', 475: '2022-05-02', 476: '2022-05-03', 477: '2022-05-04', 478: '2022-05-05', 479: '2022-05-06', 480: '2022-05-07', 481: '2022-05-08', 482: '2022-05-09', 483: '2022-05-10', 484: '2022-05-11', 485: '2022-05-12', 486: '2022-05-13', 487: '2022-05-14', 488: '2022-05-15', 489: '2022-05-16', 490: '2022-05-17', 491: '2022-05-18', 492: '2022-05-19', 493: '2022-05-20', 494: '2022-05-21', 495: '2022-05-22', 496: '2022-05-23', 497: '2022-05-24', 498: '2022-05-25', 499: '2022-05-26', 500: '2022-05-27', 501: '2022-05-28', 502: '2022-05-29', 503: '2022-05-30', 504: '2022-05-31', 505: '2022-06-01', 506: '2022-06-02', 507: '2022-06-03', 508: '2022-06-04', 509: '2022-06-05', 510: '2022-06-06', 511: '2022-06-07', 512: '2022-06-08', 513: '2022-06-09', 514: '2022-06-10', 515: '2022-06-11', 516: '2022-06-12', 517: '2022-06-13', 518: '2022-06-14', 519: '2022-06-15', 520: '2022-06-16', 521: '2022-06-17', 522: '2022-06-18', 523: '2022-06-19', 524: '2022-06-20', 525: '2022-06-21', 526: '2022-06-22', 527: '2022-06-23', 528: '2022-06-24', 529: '2022-06-25', 530: '2022-06-26', 531: '2022-06-27', 532: '2022-06-28', 533: '2022-06-29', 534: '2022-06-30', 535: '2022-07-01', 536: '2022-07-02', 537: '2022-07-03', 538: '2022-07-04', 539: '2022-07-05', 540: '2022-07-06', 541: '2022-07-07', 542: '2022-07-08', 543: '2022-07-09', 544: '2022-07-10', 545: '2022-07-11', 546: '2022-07-12', 547: '2022-07-13', 548: '2022-07-14', 549: '2022-07-15', 550: '2022-07-16', 551: '2022-07-17', 552: '2022-07-18', 553: '2022-07-19', 554: '2022-07-20', 555: '2022-07-21', 556: '2022-07-22', 557: '2022-07-23', 558: '2022-07-24', 559: '2022-07-25', 560: '2022-07-26', 561: '2022-07-27', 562: '2022-07-28', 563: '2022-07-29', 564: '2022-07-30', 565: '2022-07-31', 566: '2022-08-01', 567: '2022-08-02', 568: '2022-08-03', 569: '2022-08-04', 570: '2022-08-05', 571: '2022-08-06', 572: '2022-08-07', 573: '2022-08-08', 574: '2022-08-09', 575: '2022-08-10', 576: '2022-08-11', 577: '2022-08-12', 578: '2022-08-13', 579: '2022-08-14', 580: '2022-08-15', 581: '2022-08-16', 582: '2022-08-17', 583: '2022-08-18', 584: '2022-08-19', 585: '2022-08-20', 586: '2022-08-21', 587: '2022-08-22', 588: '2022-08-23', 589: '2022-08-24', 590: '2022-08-25', 591: '2022-08-26', 592: '2022-08-27', 593: '2022-08-28', 594: '2022-08-29', 595: '2022-08-30', 596: '2022-08-31', 597: '2022-09-01', 598: '2022-09-02', 599: '2022-09-03', 600: '2022-09-04', 601: '2022-09-05', 602: '2022-09-06', 603: '2022-09-07', 604: '2022-09-08', 605: '2022-09-09', 606: '2022-09-10', 607: '2022-09-11', 608: '2022-09-12', 609: '2022-09-13', 610: '2022-09-14', 611: '2022-09-15', 612: '2022-09-16', 613: '2022-09-17', 614: '2022-09-18', 615: '2022-09-19', 616: '2022-09-20', 617: '2022-09-21', 618: '2022-09-22', 619: '2022-09-23', 620: '2022-09-24', 621: '2022-09-25', 622: '2022-09-26', 623: '2022-09-27', 624: '2022-09-28', 625: '2022-09-29', 626: '2022-09-30', 627: '2022-10-01', 628: '2022-10-02', 629: '2022-10-03', 630: '2022-10-04', 631: '2022-10-05', 632: '2022-10-06', 633: '2022-10-07', 634: '2022-10-08', 635: '2022-10-09', 636: '2022-10-10', 637: '2022-10-11', 638: '2022-10-12', 639: '2022-10-13', 640: '2022-10-14', 641: '2022-10-15', 642: '2022-10-16', 643: '2022-10-17', 644: '2022-10-18', 645: '2022-10-19', 646: '2022-10-20', 647: '2022-10-21', 648: '2022-10-22', 649: '2022-10-23', 650: '2022-10-24', 651: '2022-10-25', 652: '2022-10-26', 653: '2022-10-27', 654: '2022-10-28', 655: '2022-10-29', 656: '2022-10-30', 657: '2022-10-31', 658: '2022-11-01', 659: '2022-11-02', 660: '2022-11-03', 661: '2022-11-04', 662: '2022-11-05', 663: '2022-11-06', 664: '2022-11-07', 665: '2022-11-08', 666: '2022-11-09', 667: '2022-11-10', 668: '2022-11-11', 669: '2022-11-12', 670: '2022-11-13', 671: '2022-11-14', 672: '2022-11-15', 673: '2022-11-16', 674: '2022-11-17', 675: '2022-11-18', 676: '2022-11-19', 677: '2022-11-20', 678: '2022-11-21', 679: '2022-11-22', 680: '2022-11-23', 681: '2022-11-24', 682: '2022-11-25', 683: '2022-11-26', 684: '2022-11-27', 685: '2022-11-28', 686: '2022-11-29', 687: '2022-11-30', 688: '2022-12-01', 689: '2022-12-02', 690: '2022-12-03', 691: '2022-12-04', 692: '2022-12-05', 693: '2022-12-06', 694: '2022-12-07', 695: '2022-12-08', 696: '2022-12-09', 697: '2022-12-10', 698: '2022-12-11', 699: '2022-12-12', 700: '2022-12-13', 701: '2022-12-14', 702: '2022-12-15', 703: '2022-12-16', 704: '2022-12-17', 705: '2022-12-18', 706: '2022-12-19', 707: '2022-12-20', 708: '2022-12-21', 709: '2022-12-22', 710: '2022-12-23', 711: '2022-12-24', 712: '2022-12-25', 713: '2022-12-26', 714: '2022-12-27', 715: '2022-12-28', 716: '2022-12-29', 717: '2022-12-30', 718: '2022-12-31', 719: '2023-01-01', 720: '2023-01-02', 721: '2023-01-03', 722: '2023-01-04', 723: '2023-01-05', 724: '2023-01-06', 725: '2023-01-07', 726: '2023-01-08', 727: '2023-01-09', 728: '2023-01-10', 729: '2023-01-11', 730: '2023-01-12', 731: '2023-01-13', 732: '2023-01-14', 733: '2023-01-15', 734: '2023-01-16', 735: '2023-01-17', 736: '2023-01-18', 737: '2023-01-19', 738: '2023-01-20', 739: '2023-01-21', 740: '2023-01-22', 741: '2023-01-23', 742: '2023-01-24', 743: '2023-01-25', 744: '2023-01-26', 745: '2023-01-27', 746: '2023-01-28', 747: '2023-01-29', 748: '2023-01-30', 749: '2023-01-31', 750: '2023-02-01', 751: '2023-02-02', 752: '2023-02-03', 753: '2023-02-04', 754: '2023-02-05', 755: '2023-02-06', 756: '2023-02-07', 757: '2023-02-08', 758: '2023-02-09', 759: '2023-02-10', 760: '2023-02-11', 761: '2023-02-12', 762: '2023-02-13', 763: '2023-02-14', 764: '2023-02-15', 765: '2023-02-16', 766: '2023-02-17', 767: '2023-02-18', 768: '2023-02-19', 769: '2023-02-20', 770: '2023-02-21', 771: '2023-02-22', 772: '2023-02-23', 773: '2023-02-24', 774: '2023-02-25', 775: '2023-02-26', 776: '2023-02-27', 777: '2023-02-28', 778: '2023-03-01', 779: '2023-03-02', 780: '2023-03-03', 781: '2023-03-04', 782: '2023-03-05', 783: '2023-03-06', 784: '2023-03-07', 785: '2023-03-08', 786: '2023-03-09', 787: '2023-03-10', 788: '2023-03-11', 789: '2023-03-12', 790: '2023-03-13', 791: '2023-03-14', 792: '2023-03-15', 793: '2023-03-16', 794: '2023-03-17', 795: '2023-03-18', 796: '2023-03-19', 797: '2023-03-20', 798: '2023-03-21', 799: '2023-03-22', 800: '2023-03-23', 801: '2023-03-24', 802: '2023-03-25', 803: '2023-03-26', 804: '2023-03-27', 805: '2023-03-28', 806: '2023-03-29', 807: '2023-03-30', 808: '2023-03-31', 809: '2023-04-01', 810: '2023-04-02', 811: '2023-04-03', 812: '2023-04-04', 813: '2023-04-05', 814: '2023-04-06', 815: '2023-04-07', 816: '2023-04-08', 817: '2023-04-09', 818: '2023-04-10', 819: '2023-04-11', 820: '2023-04-12', 821: '2023-04-13', 822: '2023-04-14', 823: '2023-04-15', 824: '2023-04-16', 825: '2023-04-17', 826: '2023-04-18', 827: '2023-04-19', 828: '2023-04-20', 829: '2023-04-21', 830: '2023-04-22', 831: '2023-04-23', 832: '2023-04-24', 833: '2023-04-25', 834: '2023-04-26', 835: '2023-04-27', 836: '2023-04-28', 837: '2023-04-29', 838: '2023-04-30', 839: '2023-05-01', 840: '2023-05-02', 841: '2023-05-03', 842: '2023-05-04', 843: '2023-05-05', 844: '2023-05-06', 845: '2023-05-07', 846: '2023-05-08', 847: '2023-05-09', 848: '2023-05-10', 849: '2023-05-11', 850: '2023-05-12', 851: '2023-05-13', 852: '2023-05-14', 853: '2023-05-15', 854: '2023-05-16', 855: '2023-05-17', 856: '2023-05-18', 857: '2023-05-19', 858: '2023-05-20', 859: '2023-05-21', 860: '2023-05-22', 861: '2023-05-23', 862: '2023-05-24', 863: '2023-05-25', 864: '2023-05-26', 865: '2023-05-27', 866: '2023-05-28', 867: '2023-05-29', 868: '2023-05-30', 869: '2023-05-31', 870: '2023-06-01', 871: '2023-06-02', 872: '2023-06-03', 873: '2023-06-04', 874: '2023-06-05', 875: '2023-06-06', 876: '2023-06-07', 877: '2023-06-08', 878: '2023-06-09', 879: '2023-06-10', 880: '2023-06-11', 881: '2023-06-12', 882: '2023-06-13', 883: '2023-06-14', 884: '2023-06-15', 885: '2023-06-16', 886: '2023-06-17', 887: '2023-06-18', 888: '2023-06-19', 889: '2023-06-20', 890: '2023-06-21', 891: '2023-06-22', 892: '2023-06-23', 893: '2023-06-24', 894: '2023-06-25', 895: '2023-06-26', 896: '2023-06-27', 897: '2023-06-28', 898: '2023-06-29', 899: '2023-06-30', 900: '2023-07-01', 901: '2023-07-02', 902: '2023-07-03', 903: '2023-07-04', 904: '2023-07-05', 905: '2023-07-06', 906: '2023-07-07', 907: '2023-07-08', 908: '2023-07-09', 909: '2023-07-10', 910: '2023-07-11', 911: '2023-07-12', 912: '2023-07-13', 913: '2023-07-14', 914: '2023-07-15', 915: '2023-07-16', 916: '2023-07-17', 917: '2023-07-18', 918: '2023-07-19', 919: '2023-07-20', 920: '2023-07-21', 921: '2023-07-22', 922: '2023-07-23', 923: '2023-07-24', 924: '2023-07-25', 925: '2023-07-26', 926: '2023-07-27', 927: '2023-07-28', 928: '2023-07-29', 929: '2023-07-30', 930: '2023-07-31', 931: '2023-08-01', 932: '2023-08-02', 933: '2023-08-03', 934: '2023-08-04', 935: '2023-08-05', 936: '2023-08-06', 937: '2023-08-07', 938: '2023-08-08', 939: '2023-08-09', 940: '2023-08-10', 941: '2023-08-11', 942: '2023-08-12', 943: '2023-08-13', 944: '2023-08-14', 945: '2023-08-15', 946: '2023-08-16'}


Encoding key for capital_projects.soft_opening_date:
{0: '2021-01-12', 1: '2021-01-26', 2: '2021-02-02', 3: '2021-03-30', 4: '2021-04-13', 5: '2021-05-13', 6: '2021-05-18', 7: '2021-05-25', 8: '2021-06-15', 9: '2021-06-22', 10: '2021-07-13', 11: '2021-07-20', 12: '2021-07-27', 13: '2021-08-17', 14: '2021-09-21', 15: '2021-09-28', 16: '2021-10-19', 17: '2021-11-09', 18: '2021-11-17', 19: '2021-12-07', 20: '2022-03-01', 21: '2022-03-15', 22: '2022-05-03', 23: '2022-05-10', 24: '2022-05-17', 25: '2022-06-14', 26: '2022-06-28', 27: '2022-07-12', 28: '2022-08-02', 29: '2022-08-09', 30: '2022-08-16'}


In [26]:
## # Separating Predictors and Target Variables
predictors = combined_data[['capital_projects.soft_opening_date', 'calendar.calendar_day_date', 'calendar.fiscal_week_id_for_year', 'calendar.day_of_week', 'calendar_information.holiday', 'calendar_information.type_of_day', 'open_year', 'square_feet', 'front_door_count', 'years_since_last_project', 'parking_spaces', 'lottery', 'freal', 'bonfire_grill', 'pizza', 'cinnabon', 'godfather_s_pizza', 'ethanol_free', 'diesel_y', 'hi_flow_lanes', 'rv_lanes', 'hi_flow_rv_lanes', 'def', 'cat_scales', 'car_wash', 'ev_charging', 'rv_dumps', 'propane', 'x1_mile_pop', 'x1_mile_emp', 'x1_mile_income', 'x1_2_mile_pop', 'x1_2_mile_emp', 'x1_2_mile_income', 'x5_min_pop', 'x5_min_emp', 'x5_min_inc', 'x7_min_pop', 'x7_min_emp', 'x7_min_inc', 'traditional_forecourt_fueling_positions', 'traditional_forecourt_layout', 'traditional_forecourt_stack_type', 'rv_lanes_fueling_positions', 'rv_lanes_layout', 'rv_lanes_stack_type', 'hi_flow_lanes_fueling_positions', 'hi_flow_lanes_layout', 'hi_flow_lanes_stack_type', 'hi_flow_lanes_fueling_positions_2', 'rv_lanes_fueling_positions_2', 'hi_flow_rv_lanes_layout', 'hi_flow_rv_lanes_stack_type', 'non_24_hour', 'self_check_out', 'mens_toilet_count', 'mens_urinal_count', 'womens_toilet_count', 'womens_sink_count']]
target_variables = combined_data[['daily_yoy_ndt.total_inside_sales', 'daily_yoy_ndt.total_food_service', 'diesel_x', 'unleaded']]

# Splitting the Data into Test and Training Sets
predictors_train, predictors_test, target_variables_train, target_variables_test = train_test_split(predictors, target_variables, test_size=0.2, random_state=42)

# Function to train SARIMA models
def train_sarima_models(target_variables_train, predictors_train):
    sarima_models = {}

    for target_variable in target_variables_train.columns:
        # Perform a grid search for the best hyperparameters
        best_model = auto_arima(
            target_variables_train[target_variable],
            exogenous=predictors_train,
            seasonal=True,
            m=7,
            stepwise=True,
            trace=True,
            start_p=0, start_q=0,
            max_p=3, max_q=3,
            start_P=0, start_Q=0,
            max_P=3, max_Q=3,
            suppress_warnings=True,
            error_action="ignore"
        )

        # Retrieve the best hyperparameters
        best_p, best_d, best_q = best_model.get_params()['order']
        best_P, best_D, best_Q, best_S = best_model.get_params()['seasonal_order']

        # Fit the SARIMA model with the best hyperparameters
        sarima_model = SARIMAX(target_variables_train[target_variable], exog=predictors_train, order=(best_p, best_d, best_q), seasonal_order=(best_P, best_D, best_Q, best_S))
        sarima_results = sarima_model.fit()
        sarima_models[target_variable] = sarima_results

    return sarima_models

# Function to predict with SARIMA models
def predict_with_sarima_models(user_input, sarima_models, predictors_test, target_variables_test):
    # Create a DataFrame with the user's input as exogenous variables
    exog_data = pd.DataFrame(user_input, index=[0])

    # Initialize a dictionary to store the predicted values for each target variable
    forecasted_values = {}

    # Use the pre-trained SARIMA models to make predictions for the user's input
    for target_variable, model in sarima_models.items():
        try:
            if target_variable not in target_variables_train.columns:
                print(f"{target_variable} is not an outcome variable. Please provide values only for outcome variables.")
                continue
            forecasted_values[target_variable] = model.get_forecast(steps=len(target_variables_test), exog=exog_data.fillna(0))
        except ValueError:
            print(f"Invalid input for {target_variable}. Please enter a valid numerical value.")

    return forecasted_values

# Train the SARIMA models using training data
sarima_models = train_sarima_models(target_variables_train, predictors_train)

# Prompt the user to input values for predictor variables
user_input = {}
for predictor in predictors_train.columns:
    while True:  # Keep asking until a valid input is provided
        try:
            value = float(input(f"Enter the value for {predictor}: "))
            user_input[predictor] = value
            break  # Exit the loop if input is valid
        except ValueError:
            print(f"Invalid input for {predictor}. Please enter a valid numerical value.")
            # Re-prompt for valid input
            user_input[predictor] = float(input(f"Enter the value for {predictor}: "))

# Call the function to predict the store's performance based on user input
forecasted_values = predict_with_sarima_models(user_input, sarima_models, predictors_test, target_variables_test)

# Get today's date
current_date = pd.Timestamp(datetime.now())

# Generate a date range for the next year (365 days) starting from today
prediction_dates = pd.date_range(start=current_date, periods=365)

# Create exogenous variables for the next year
exog_next_year = pd.DataFrame(index=prediction_dates, columns=predictors.columns)

# Fill exog_next_year with the same data for each day
for col in exog_next_year.columns:
    exog_next_year[col] = predictors[col].values[0]

# Initialize a dictionary to store daily sales projections
daily_projections = {}

# Specify the target variables for which you want to calculate daily projections
target_variables = combined_data[['daily_yoy_ndt.total_inside_sales', 'daily_yoy_ndt.total_food_service', 'diesel_x', 'unleaded']]

# Make predictions for each of the target variables
for target_variable in target_variables:
    # Use the SARIMA model to forecast sales for the next year
    forecasted_values = sarima_models[target_variable].get_forecast(steps=365, exog=exog_next_year)

    # Extract the predicted values
    predicted_values = forecasted_values.predicted_mean

    # Store the daily projections in the dictionary and reset the index
    daily_projections[target_variable] = predicted_values
    daily_projections[target_variable].index = prediction_dates

# Print the table for daily projections for each target variable
print("Daily Projections for Target Variables:")
for target_variable, daily_projection in daily_projections.items():
    print(f"\n{target_variable}:")
    # Round the daily projections to 2 decimal places
    daily_projection_rounded = daily_projection.round(2)
    print(daily_projection_rounded)


# Create a DataFrame from daily_projections
daily_projections_df = pd.DataFrame(daily_projections)

# Round the values in the DataFrame to 2 decimal places
daily_projections_df = daily_projections_df.round(2)

# Set the date index to a column
daily_projections_df['date'] = daily_projections_df.index

# Reorder the columns to match the desired order
desired_order = [
    'daily_yoy_ndt.total_inside_sales', 'daily_yoy_ndt.total_food_service', 'diesel_x', 'unleaded'
]
daily_projections_df = daily_projections_df[desired_order]

# Export the DataFrame to a CSV file
daily_projections_df.to_csv('daily_projections.csv', index=False)

# Display the top 10 rows
print(daily_projections_df.head(365))



Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[7] intercept   : AIC=180221.667, Time=0.10 sec
 ARIMA(1,0,0)(1,0,0)[7] intercept   : AIC=180222.176, Time=2.32 sec
 ARIMA(0,0,1)(0,0,1)[7] intercept   : AIC=180219.033, Time=0.79 sec
 ARIMA(0,0,0)(0,0,0)[7]             : AIC=204374.695, Time=0.06 sec
 ARIMA(0,0,1)(0,0,0)[7] intercept   : AIC=180223.541, Time=0.30 sec
 ARIMA(0,0,1)(1,0,1)[7] intercept   : AIC=180221.393, Time=5.91 sec
 ARIMA(0,0,1)(0,0,2)[7] intercept   : AIC=180221.033, Time=1.37 sec
 ARIMA(0,0,1)(1,0,0)[7] intercept   : AIC=180224.661, Time=1.52 sec
 ARIMA(0,0,1)(1,0,2)[7] intercept   : AIC=180223.023, Time=6.29 sec
 ARIMA(0,0,0)(0,0,1)[7] intercept   : AIC=180217.204, Time=0.43 sec
 ARIMA(0,0,0)(1,0,1)[7] intercept   : AIC=180219.725, Time=5.41 sec
 ARIMA(0,0,0)(0,0,2)[7] intercept   : AIC=180219.203, Time=1.03 sec
 ARIMA(0,0,0)(1,0,0)[7] intercept   : AIC=180222.972, Time=0.57 sec
 ARIMA(0,0,0)(1,0,2)[7] intercept   : AIC=180221.204, Time=5.10 sec
 ARIM

C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[7] intercept   : AIC=157317.359, Time=0.10 sec
 ARIMA(1,0,0)(1,0,0)[7] intercept   : AIC=157318.547, Time=1.54 sec
 ARIMA(0,0,1)(0,0,1)[7] intercept   : AIC=157316.165, Time=0.71 sec
 ARIMA(0,0,0)(0,0,0)[7]             : AIC=176647.597, Time=0.06 sec
 ARIMA(0,0,1)(0,0,0)[7] intercept   : AIC=157319.324, Time=0.29 sec
 ARIMA(0,0,1)(1,0,1)[7] intercept   : AIC=157323.860, Time=4.90 sec
 ARIMA(0,0,1)(0,0,2)[7] intercept   : AIC=157318.117, Time=1.53 sec
 ARIMA(0,0,1)(1,0,0)[7] intercept   : AIC=157320.480, Time=1.31 sec
 ARIMA(0,0,1)(1,0,2)[7] intercept   : AIC=162590.286, Time=15.14 sec
 ARIMA(0,0,0)(0,0,1)[7] intercept   : AIC=157314.193, Time=0.38 sec
 ARIMA(0,0,0)(1,0,1)[7] intercept   : AIC=157320.520, Time=5.59 sec
 ARIMA(0,0,0)(0,0,2)[7] intercept   : AIC=157316.145, Time=1.07 sec
 ARIMA(0,0,0)(1,0,0)[7] intercept   : AIC=157318.485, Time=0.70 sec
 ARIMA(0,0,0)(1,0,2)[7] intercept   : AIC=157485.799, Time=12.50 sec
 AR

C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[7] intercept   : AIC=197488.475, Time=0.10 sec
 ARIMA(1,0,0)(1,0,0)[7] intercept   : AIC=197481.627, Time=0.77 sec
 ARIMA(0,0,1)(0,0,1)[7] intercept   : AIC=197477.468, Time=0.64 sec
 ARIMA(0,0,0)(0,0,0)[7]             : AIC=202783.240, Time=0.06 sec
 ARIMA(0,0,1)(0,0,0)[7] intercept   : AIC=197490.472, Time=0.25 sec
 ARIMA(0,0,1)(1,0,1)[7] intercept   : AIC=197479.451, Time=4.84 sec
 ARIMA(0,0,1)(0,0,2)[7] intercept   : AIC=197479.141, Time=1.42 sec
 ARIMA(0,0,1)(1,0,0)[7] intercept   : AIC=197482.716, Time=0.70 sec
 ARIMA(0,0,1)(1,0,2)[7] intercept   : AIC=197481.143, Time=9.00 sec
 ARIMA(0,0,0)(0,0,1)[7] intercept   : AIC=197475.480, Time=0.42 sec
 ARIMA(0,0,0)(1,0,1)[7] intercept   : AIC=197477.483, Time=2.96 sec
 ARIMA(0,0,0)(0,0,2)[7] intercept   : AIC=197477.154, Time=1.22 sec
 ARIMA(0,0,0)(1,0,0)[7] intercept   : AIC=197481.200, Time=0.55 sec
 ARIMA(0,0,0)(1,0,2)[7] intercept   : AIC=197479.165, Time=9.68 sec
 ARIM

C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[7] intercept   : AIC=181323.524, Time=0.12 sec
 ARIMA(1,0,0)(1,0,0)[7] intercept   : AIC=181324.729, Time=1.41 sec
 ARIMA(0,0,1)(0,0,1)[7] intercept   : AIC=181324.313, Time=0.66 sec
 ARIMA(0,0,0)(0,0,0)[7]             : AIC=201201.598, Time=0.06 sec
 ARIMA(0,0,0)(1,0,0)[7] intercept   : AIC=181325.358, Time=1.32 sec
 ARIMA(0,0,0)(0,0,1)[7] intercept   : AIC=181324.476, Time=0.42 sec
 ARIMA(0,0,0)(1,0,1)[7] intercept   : AIC=181330.213, Time=4.53 sec
 ARIMA(1,0,0)(0,0,0)[7] intercept   : AIC=181323.284, Time=0.22 sec
 ARIMA(1,0,0)(0,0,1)[7] intercept   : AIC=181324.238, Time=0.86 sec
 ARIMA(1,0,0)(1,0,1)[7] intercept   : AIC=181329.409, Time=7.69 sec
 ARIMA(2,0,0)(0,0,0)[7] intercept   : AIC=181325.121, Time=0.31 sec
 ARIMA(1,0,1)(0,0,0)[7] intercept   : AIC=181325.288, Time=0.36 sec
 ARIMA(0,0,1)(0,0,0)[7] intercept   : AIC=181323.358, Time=0.25 sec
 ARIMA(2,0,1)(0,0,0)[7] intercept   : AIC=181327.121, Time=0.71 sec
 ARIM

C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Enter the value for capital_projects.soft_opening_date: 18
Enter the value for calendar.calendar_day_date: 926
Enter the value for calendar.fiscal_week_id_for_year: 22
Enter the value for calendar.day_of_week: 4
Enter the value for calendar_information.holiday: 17
Enter the value for calendar_information.type_of_day: 0
Enter the value for open_year: 2022
Enter the value for square_feet: 3500
Enter the value for front_door_count: 2
Enter the value for years_since_last_project: 2
Enter the value for parking_spaces: 15
Enter the value for lottery: 1
Enter the value for freal: 1
Enter the value for bonfire_grill: 0
Enter the value for pizza: 1
Enter the value for cinnabon: 1
Enter the value for godfather_s_pizza: 0
Enter the value for ethanol_free: 1
Enter the value for diesel_y: 1
Enter the value for hi_flow_lanes: 1
Enter the value for rv_lanes: 1
Enter the value for hi_flow_rv_lanes: 1
Enter the value for def: 1
Enter the value for cat_scales: 1
Enter the value for car_wash: 0
Enter the

C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\jusha\New folder\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\jusha\Ne

Because the model is built the same way as for when we run it against a CSV file, we are getting the same performance metrics. However in this code chunk we allow the user to input unique values and use the model to experiment with values of the predictors. The model code above runs using a CSV, but we have similar results. This code chunk just allows the models to run against the users input to experiment for a certain store. The model performance metrics remain the same because they are against the grid search of the hyper parameters and the same test and training datasets. This code simply is more user friendly rather than being dependent on running the model against a CSV File. 


We still depend on using the model to run against a CSV file as that will be faster to find an average for each target variable for the stores, but this is a helpful tool that can be used. For the input variables if a customer inserts a value that is not accepted, such as a 2 in a categorical variable value that is not accepted then an error will occur with an invalid input. The model will also default to a 0 for some categorical variables if the number inserted is not a categorical variable input option. The custom output is then saved and printed into the users computer as a CSV file that can be pulled up and viewed in more detail. As The store runs with new data, then as time goes on then the predictions will become more accurate with the sales of the store. 


This will allow the user to input customer variables for the model to predict based on the information for each store, then the model will predict sales for a store in that situation. The input allows us to see what the daily years sales predicted are going to be a store with specific inputted values. Once again, the M value would need to be changed to 365 for daily predictions when used in a powerful computer. This development would require > 15GB of RAM. 

This is helpful to see the individual sales projections of the inside sales, food service, diesel, and unleaded fuel based on customer inputted values of the stores. While the model does accept the date as a predictor, the model will then take and make the predictions with the automated current date. The full predictions would need to be accessed through your local computer destination. 


# Model Performance

From our SARIMA model, we do use grid search to fine tune the hyper parameters for each of the SARIMA Models. We use a For Loop to automate the development of the model for the model that predicts the daily sales of diesel, unleaded, inside sales, and food service sales within the year. Thus it creates a different model for each. 

As far as performance of timing and memory usage, this is the trick. We know that if we ran a model where we set our seasonality or m = to 365, we could get better results that would predict the daily sales throughout the course of the year. However, this type of model took about 4 hours to run and takes 12 GB of RAM to perform. Depending on the power of a computer, this model could be ran more efficiently, but the computer that I was using only has 8 GB of RAM, so there was a constant MemoryError and the model when M = 365 was not able to run.  To overcome this, I set M = 7 which would pick up on the first weeks patterns of seasonality for each date the transaction occur, how ever when we print the results of the predictions we only get the first 7 predictions to be valuable while the remainder in the year are not able to be fully seen because the model to predict 365 days could not be executed due to the lack of Memory. 

When we use multiple values of m, we do see that our RMSE and R2 remain constant despite the changes to Seasonality. That being said that when you increase the M = 365, you will only get more accurate values in the outputted table of predicted values for the stores location. The performance of the model when predicted will remain the same despite changes to M increasing. It is cautioned that increase M by too much may potentially lead to over-fitting in the model, but the case is more seen that the increase in M Values only improved R Squared and RMSE values. That being said, we can use the model when M = 7 to derive with how the model itself performs. The performance of each of the models for Total Inside Sales, Food Service Sales, Diesel Sales, and Unleaded Sales can be broken down as:



* Modeling the Daily Year over Year Total Inside Sales
    * Best Hyper Parameters for Tuning (When M = 7, could change with more powerful Computer When M = 365):
      * ARIMA(0,0,0)(0,0,1)[7] intercept
       * Total fit time: 76.753 seconds
    * Root Mean Squared Error for daily_yoy_ndt.total_inside_sales: 624.7390227242362
        * Interpreted: This means that our models predictions for the Total Inside Sales is going to be off by about 624.74 units (in this case dollar sales). This is may at first appear like a large difference, but in the total sales of the day this could be fairly small. 
    * R-squared for daily_yoy_ndt.total_inside_sales: 0.5904968531834881
        * Interpreted this means that our model is explaining about 59% of the variance in total inside sales from the predictors in our model. Although this metric could be improved, because this is what our algorithm has chosen as the best possible hyper parameters and tuning this explanation is still pretty good. Because we are past the point of 50% we know that our model is performing better compared to simply guessing what the amount may be. 
        
* Modeling the Daily Year over Year Total Food Service Sales
    * Best Hyper Parameters for Tuning (When M = 7, could change with more powerful Computer When M = 365):
         * Best model:  ARIMA(0,0,0)(0,0,1)[7] intercept
        * Total fit time: 110.598 seconds
    * Root Mean Squared Error for daily_yoy_ndt.total_food_service: 191.09674129029995
        *  Interpreted: This means that our models predictions for the Total Food Service Sales is going to be off by about 191.10 units (In this case dollar bills of sales). Considering this is only the difference of potentially 19 pizzas priced at about 10 dollars, this RMSE is very good and does describe a great picture of what we can expect our stores food service sales to be.
    * R-squared for daily_yoy_ndt.total_food_service: 0.6777146253101345
        * Interpreted: This means that 67.78% of the variance of total amount sold by food service is explained by the model with the best hyper parameters. This is a good R Squared because it shows us that our model is able to explain much of the variance of the total amount spent on food service at the Maverik location. This is an impressive measurement.

* Modeling the Daily Diesel Sales
    * Best Hyper Parameters for Tuning (When M = 7, could change with more powerful Computer When M = 365):
      * Best model:  ARIMA(0,0,0)(0,0,1)[7] intercept
       * Total fit time: 79.172 seconds
    * Root Mean Squared Error for diesel_x: 947.0812310686053
        *  Interpreted: This means that our models predictions for the units of diesel sold is going to be off by about 947 units (In this case, gallons). Put into context a typical semi truck can hold an average of about 120-150 gallons of diesel fuel. Meaning that our model prediction may be about 6 or 7 semi truck fill ups off from the actual value of that day. This is highly impressive. 
    * R-squared for diesel_x: 0.795214411020868
        * Interpreted: This means that 79.52% of the variance in the amount of diesel gallons distributed at a location each day is explained by our model. This is an exceptional R Squared value because it shows that the model is explaining a large majority of the variance for diesel fuel sold, but it is not too high that we should be concerned about over-fitting from the model. We can be confident in the values that the Model predicts. 


* Modeling for the Unleaded Fuel Sales 
    * Best Hyper Parameters for Tuning (When M = 7, could change with more powerful Computer When M = 365):
      * Best model:  ARIMA(1,0,0)(0,0,0)[7] intercept
       * Total fit time: 41.251 seconds
    * Root Mean Squared Error for unleaded: 524.6061091878332
        * Interpreted: This means that our models predictions for the units of unleaded fuel sold is going to be off by about 525 units (In this case, gallons). Put into context a typical car model can hold an average of about 10-18 gallons of unleaded traditional gasoline fuel. Meaning that our model prediction may be about 29 or 53 car fill ups off from the actual value of that day. This is highly impressive. This is also a fairly small difference considering the amount of cars that go through a gas station pump to fill up each day. 
    * R-squared for unleaded: 0.7288772237840739
        * Interpreted: This means that 72.89% of the variance in the amount of unleaded fuel gallons distributed at a location each day is explained by our model. This is an exceptional R Squared value because it shows that the model is explaining a large majority of the variance for unleaded fuel gallons sold, but it is not too high that we should be concerned about over-fitting from the model. We can be confident in the values that the Model predicts.

# Results

# Group Member Contributions